# Exercise Sheet 4: Machine Learning Fundamentals & Linear Regression (Deadline: 01 Dec 23:59)

# ML Fundamentals(7 points)
For theoretical tasks you are encouraged to write in $\\LaTeX$. Jupyter notebooks support them by default. For reference, please have a look at the examples in this short excellent guide: [Typesetting Equations](http://nbviewer.jupyter.org/github/ipython/ipython/blob/3.x/examples/Notebook/Typesetting%20Equations.ipynb)

Alternatively, you can upload the solutions in the written form as images and paste them inside the cells. But if you do this, **make sure** that the images are of high quality, so that we can read them without any problems.

###### 1. Sigmoid Function (1.5 points)
The special case of the logistic function is the *sigmoid function* which is defined as:

\begin{equation*}
  \sigma(a) = \frac{1}{1 + e^{-a}}
\end{equation*}

a) Compute its gradient analytically. (0.5 points)

b) What are the inherent properties that you observe from the above computed gradient? (0.5 points) <br />
   *Hint: Think about how would the gradient signal be for the whole domain of the sigmoid function*

c) Prove that the sigmoid function is symmetric. (0.5 points)

###### 2. Regularization (3.5 points)

In the lecture, we've seen that we can add a *regularizer* to our cost function to avoid *over or underfitting*. For example, consider the following training criterion for linear regression:

\begin{equation*}
  J(\textbf{w}) = \frac{1}{m}\sum_{i=1}^{m} \Vert\hat{y}^{(i)} - y^{(i)}\Vert^{2} + \lambda\Omega(\textbf{w})
\end{equation*}
where $\Omega(\textbf{w}) = \textbf{w}^{T}\textbf{w}$ is the regularizer.

a) In the above criterion, what is the role of the regularization parameter $\lambda$ on the regularizer (i.e. parameters of our model) while minimizing $J(\textbf{w})$? (1.0 point)

b) Is $\lambda$ the model parameter or a hyperparameter? Justify.(0.5 points)

c) Derive the closed form solution for the weights ($\textbf{w}$) in the above criterion.(2.0 points)

###### 3. Maximum Likelihood Estimation (MLE) (2 points)
Consider the density function of a ***univariate Gaussian distribution***


\begin{equation*}
 p(x;\mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}exp\left(-\frac{1}{2\sigma^2}(x-\mu)^{2}\right)
\end{equation*}
where $\mu$ is the $\textit{mean}$ and $\sigma^{2}$ is the $\textit{variance}$. 

Let's say you're given *N* samples (i.e. $x_1, x_2, x_3, ..., x_N$) which are drawn from the above stated distribution. Also, you can assume that these samples are **i.i.d** (i.e. [independent and identically distributed](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)).

Now, please derive the *MLE step-by-step* for:

a) *mean* $(\mu)$. (1.0 point)

b) *variance* $(\sigma^2)$. (1.0 point)

# Multiple Linear Regression (13 points)

#### 1. Introduction
As we have seen in first assignment sheet, when we have one independent (or explanatory) variable and a scalar dependent variable, it is called **simple linear regression**.
But, when there are more than one explanatory variable (i.e. $x^{(1)}, x^{(2)}, ...,x^{(k)}$), and a single scalar dependent variable (*y*), then it's called $\textit{multiple linear regression}$. (Please don't confuse this with *multivariate linear regression* where we predict more than one (correlated) dependent variable.)

Here, we will implement a **multiple linear regression** model in Python/NumPy using the *Gradient Descent* algorithm. Particularly, we will be using $\textit{stochastic gradient descent}$ (*SGD*) where one performs the update step using a small set of training samples of size *batch_size* which we will set to 64. This is again a hyperparameter but in this exercise we will just use a fixed batch-size of *64* (i.e. we go through the training samples sampling 64 at a time and perform gradient descent.) Such a procedure is sometimes called *mini-batch gradient descent* in the deep learning community.

Going through all the training samples *once* is called an **epoch**. Ideally, the algorithm has to go through multiple epochs over the training samples, each time shuffling it, until a convergence criterion has been satisfied. <br />

Here, we will set a *tolerance value* for the difference in error (i.e. change in MSE values between subsequent epochs) that we will accept. Once this difference falls below the *tolerance value*, we terminate our training phase and return the parameters. 

We repeat the above training procedure for all possible hyperparameter combinations. Later on, using these parameters (*i.e. weight vectors*), we compute the prediction for validation data and the corresponding MSE values. And then, we pick the hyperparameter combination which yielded the least MSE.

As a next step, we will combine training data and validation data and make it as our *new training data*. We keep the test data as it is. Using the hyperparameter combination (for the least MSE) that we found above, we train the model again with the *new training data* and obtain the parameter (*i.e. weight vector*) after convergence according to our *tolerance value*.

Phew! That will be our much desired *weight vector*. This is then used on the *test data*, which has not been seen by our algorithm so far, to make a prediction. The resulting MSE value will be the so-called [*generalization error*](https://en.wikipedia.org/wiki/Generalization_error).

It is this *generalization error* that we want it to be as low as possible for *unseen data* (implies that we can achieve higher accuracy).

#### 2. Dataset
For our task, we will be using the *Wine Quality* dataset and predict the quality of white wine based on 11 features such as acidity, citric acid content, residual sugar etc. .

In [86]:
%matplotlib inline
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# get data
data_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
data = pd.read_csv(data_url, sep=';')

# inspect data
print(data.head())
#print(data.shape)

# data as np array
data_npr = data.values

print(data_npr)

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6 

#### 3. Loss function
We will use a *regularized* form of the MSE loss function. In matrix form it can be written as follows:

\begin{equation*}
    J(\textbf{w}) = \frac{1}{2} \Vert{X\textbf{w}-\textbf{y}}\Vert^{2} + \frac{\lambda}{2}\Vert{\textbf{w}}\Vert^{2}
\end{equation*}

It's important to note that, in the above equation, $X$, called *design matrix*, is the horizontal concatenation of shape *(batch_size, num_features)* according to the *order* of the polynomial. To make things easier, you can add the *bias* term as the first column of $X$. Take care to have the *weight* vector $\textbf{w}$ with matching dimensions.

$\textit{Hint}$: see [Design_matrix#Multiple_regression](https://en.wikipedia.org/wiki/Design_matrix#Multiple_regression) for how $X$ with 2 features looks like for $1^{st}$ degree polynomial.

a) Derive the gradient (w.r.t $\textbf{w}$) for the regularized loss function given in **3**. (1.0 point)

\begin{equation*}
    J(\textbf{w}) = \frac{1}{2m} ({X\textbf{w}-\textbf{y}})^{T}({X\textbf{w}-\textbf{y}}) + \frac{\lambda}{2m}\sum_{j=1}^{n}{\textbf{w}_j}^{2}
\end{equation*}
\begin{equation*}
\nabla_wJ(\textbf{w}) = X^T({X\textbf{w}-\textbf{y}})+ \lambda\textbf{w}
\end{equation*}

#### 4. Matrix format for higher order polynomial

Written in matrix form, linear regression model for second order would look like: <br />
$$\hat{\textbf{y}} = X\textbf{w}_{1} + X^{2}\textbf{w}_{2} + \textbf{b}$$

where $X^{2}$ is the element-wise squaring of the original design matrix $X$, $\textbf{w}_1$ and $\textbf{w}_2$ are the *weight* vectors, and **b** is the *bias* vector.

a) Now, please write down the matrix format for a $9^{th}$ order linear regression model (0.5 points)
$$\hat{\textbf{y}} =\textbf{b} + \sum_{i=1}^{9} w_{i}x^{i} $$

#### 5. Hyperparameters
we will experiment with three hyperparameters:

i) regularization parameter $\lambda$ <br />
ii) learning rate $\epsilon$ <br />
iii) order of polynomial *p*

And do a grid search over the values that these hyperparameters can take in order to select the best combination (i.e. the one that achieves lowest test error). This approach is called **hyperparameter optimization or tuning**.

In [71]:
polynomial_order = [1, 5, 9]
learning_rates = [1e-5, 1e-8]
lambdas = [0.1, 0.8]

#hyperparams combination
comb_gen = itertools.product(*(polynomial_order, learning_rates, lambdas))
hparams_comb = list(comb_gen)

batch_size = 64

#### 6. Normalization
First of all, inspect the data, and understand its structure and features. Ideally, before starting to train our learning algorithm, we would want the data to be normalized. Here, we normalize the data (i.e. normalize each column) using the formula:

\begin{equation*}
  norm\_x_i = \frac{x_i - min(x)}{max(x) - min(x)}
\end{equation*}
where $x_i$ is the $i^{th}$ sample in feature $x$

a) Complete the following function which performs normalization (i.e. normalizes columns of $X$). (0.5 points)

In [75]:
def data_normalization(data):
    # TODO: implement
    newDF = pd.DataFrame() #creates a new dataframe that's empty
    for column in data:
        min_col = np.amin(data[column])
        max_col = np.amax(data[column])
        new_col = (data[column] - min_col)/(max_col - min_col)
        newDF = pd.concat([newDF, new_col], axis=1) 
    
    data_normalized = newDF.values
    return data_normalized

# perform data normalization
data_normalized = data_normalization(data)
data_npr = data_normalized
print (data_npr)

[[ 0.30769231  0.18627451  0.21686747 ...,  0.26744186  0.12903226  0.5       ]
 [ 0.24038462  0.21568627  0.20481928 ...,  0.31395349  0.24193548  0.5       ]
 [ 0.41346154  0.19607843  0.24096386 ...,  0.25581395  0.33870968  0.5       ]
 ..., 
 [ 0.25961538  0.15686275  0.11445783 ...,  0.27906977  0.22580645  0.5       ]
 [ 0.16346154  0.20588235  0.18072289 ...,  0.18604651  0.77419355
   0.66666667]
 [ 0.21153846  0.12745098  0.22891566 ...,  0.11627907  0.61290323  0.5       ]]


In [76]:
def split_data(data_npr):
    # (in-place) shuffling of data_npr along axis 0
    np.random.shuffle(data_npr)

    n_tr = 3898
    n_va = n_tr + 500
    n_te = n_va + 500
    
    X_train = data_npr[0:n_tr, 0:-1]
    Y_train = data_npr[0:n_tr, -1]
    
    X_val = data_npr[n_tr:n_va, 0:-1]
    Y_val = data_npr[n_tr:n_va, -1]
    
    X_test = data_npr[n_va:, 0:-1]
    Y_test = data_npr[n_va:, -1]
    
    return [(X_train, Y_train), (X_val, Y_val), (X_test, Y_test)]


# shuffle only the training data along axis 0
def shuffle_train_data(X_train, Y_train):
    """called after each epoch"""
    perm = np.random.permutation(len(Y_train))
    Xtr_shuf = X_train[perm]
    Ytr_shuf = Y_train[perm]
    
    return Xtr_shuf, Ytr_shuf

###### 7. Implementation of required functions

Complete the following function which computes the MSE value. (0.5 point) <br />
(i.e. just a vanilla version of it.) That is, you can ignore the regularization term and also the constants $\frac{1}{2}$

In [106]:
def compute_mse(prediction, ground_truth):
    # TODO: implement
    residual = np.subtract(prediction, ground_truth)
    squared = residual**2
    sum_of_squared = np.sum(squared)
    mse = (1/ground_truth.size)*sum_of_squared
    return mse

prediction = np.array([1, 3, 6])
truth = np.array([0, 0, 0])
mse = compute_mse(prediction, truth)
print (mse)

15.3333333333


Implement a function which computes the prediction of your model. (0.5 point)

In [107]:
def get_prediction(X, W):
    # TODO: implement
    Yhat = np.matmul(X, W)
    return Yhat

dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.4, 0.8]
for row in dataset:
    yhat = get_prediction(dataset, np.transpose(coef))
    
print (yhat)

[ 1.2  3.2  4.   2.8  6. ]


Implement a function which computes the gradient of your loss function. (1.0 point) <br />
*Hint: Just implementing the gradient computed in **3.** (a)*

In [120]:
def compute_gradient(X, Y, Yhat, W, lambda_):
    # TODO: implement
    X_T = np.transpose(X)
    pre = np.matmul(X,W)
    gradient = X_T*(pre - Y) + W * lambda_
    return gradient

X = np.array([1, 1])
coef = np.transpose(np.array([0.4, 0.8]))
truth = np.array([1])
Yhat = np.array([1.2])
print (compute_gradient(X, 1.0, Yhat, coef, 1.0))


[ 0.6  1. ]


Implement a function which performs a single update step of SGD. (0.5 point)

In [80]:
# Hint: avoid in-place modification
def sgd(gradient, lr, cur_W):
    # TODO: implement
    new_W = cur_W - (lr * gradient)
    return new_W

Complete the following function which reformats your data as a design matrix. (0.5 point)

In [94]:
# concatenate X acc. to order of polynomial; likewise do it for W
# where X is design matrix, W is the corresponding weight vector
# [1 X X^2 X^3], [1 W1 W2 W3].T

#this function is not done yet
def prepare_data_matrix(X, W, order):
    # TODO: implement
    for i in range(0, order+1):
        if(i==0):
            X_mat = np.ones(X.shape)
            W_vec = np.ones(W.shape)
        else:
            X_mat = np.concatenate((X_mat, X**(i)), axis=1)
            W_vec = np.concatenate((W_vec, W**(i)))
    
#     X_mat = None
    W_vec = np.transpose(W_vec)
    return X_mat, W_vec


# X = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
# W = np.array([[1, 2, 3]], np.int32)
# x_mat, w_vec = prepare_data_matrix(X, W, 3)
# print (x_mat)
# print (w_vec)
# prediction = get_prediction(x_mat, w_vec)

###### 8. Training
Complete the code in the following cell such that it performs **mini-batch gradient descent** on the training data for all possible hyperparameter combinations. (4.0 points)

Note: You can also define a function, named appropriately, which performs training. But, take care to do correct bookkeeping of hyperparameter combinations, weight vectors, and the MSE values.

In [129]:
splits = split_data(data_npr)
X_train, Y_train, X_val, Y_val, X_test, Y_test = itertools.chain(*splits)

tolerance = 1e-3
start = 1

# initialize weight vector from normal distribution
# TODO: implement
w_shape = X_train.shape[1]
W_init = np.random.randn(w_shape)

# cache weights for each hyperparam combination
# TODO: implement
weights_hist = {}
for order in polynomial_order:
    for lr in learning_rates:
        for lamb in lambdas:
            weights_hist[(order, lr, lamb)] = W_init

# keep track of MSE for each hparam combination. will be useful for plotting
# TODO: implement
mse_hist = {}
for order in polynomial_order:
    for lr in learning_rates:
        for lamb in lambdas:
            mse_hist[(order, lr, lamb)] = 0.0

# find optimal hyperparameters
for order in polynomial_order:
    for lr in learning_rates:
        for lamb in lambdas:
            # initialize necessary stuffs
            # TODO: implement
            W = weights_hist[(order, lr, lamb)]
            
            # design matrix needed at this point
            # use the function that we defined above
            # TODO: implement
            X_mat, W_vec = prepare_data_matrix(X_train, W, order)

            epochs = 1
            # goes through multiple epochs
            while True:
                # good idea to shuffle the train data
                # TODO: implement
                X_mat, Y_train = shuffle_train_data(X_mat, Y_train)
                
                # some more initialization
                # TODO: implement
                bs = 0
                nsamples = X_train.shape[0]
                prediction = np.empty(Y_train.shape)
                # goes through 1 epoch
                while bs < nsamples:
                    x = X_mat[bs]
                    prediction[bs] = get_prediction(x, W_vec)
                    gradient = compute_gradient(x, Y_train[bs], prediction[bs], W_vec, lamb)
                    W_vec = sgd(gradient, lr, W_vec)
#                     print("prediction: {} , ground truth: {} ".format(prediction[bs], Y_train[bs]))
                    bs = bs + 1
                    # complete code for 1 epoch
                    # TODO: implement
                    
                # after each epoch
                # get prediction for whole X_train
                # compute the MSE
                # might need to do bookkeeping of mse values as well
                mse = compute_mse(prediction, Y_train)
                mse_hist[(order, lr, lamb)] = mse

                # stopping/convergence criterion
                # check whether diff-in-mse < tolerance
                # TODO: implement
                if(mse < tolerance):
                    break
                epochs += 1
                weights_hist[(order, lr, lamb)] = W_vec
                    # cache weight vector for later use
                    # but we also need the hparam combination
                    # TODO: implement
                print("order: {} , learning rate: {} , regularizer: {} ".format(order, lr, lamb))
                print("Convergence after epoch {} with MSE {}".format(epochs, mse), "\n")
            

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 2 with MSE 28.325647337410512 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 3 with MSE 1.4556499732824661 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 4 with MSE 1.0431315315464287 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 5 with MSE 0.9951038571454035 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 6 with MSE 0.9547749920055263 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 7 with MSE 0.9550365134463659 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8 with MSE 0.9484224551681265 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 9 with MSE 0.9482105680607978 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 10 with MSE 0.9354620385251489 

order: 1 , learning rate: 1

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 81 with MSE 0.804424706620353 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 82 with MSE 0.794475756612034 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 83 with MSE 0.8110279569885431 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 84 with MSE 0.8034430274990151 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 85 with MSE 0.8106210173291772 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 86 with MSE 0.7974785783434499 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 87 with MSE 0.7965098025994953 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 88 with MSE 0.8029498557493399 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 89 with MSE 0.8040051619935702 

order: 1 , learning r

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 160 with MSE 0.7658145205989733 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 161 with MSE 0.7594663424850554 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 162 with MSE 0.7644450116352363 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 163 with MSE 0.7541039411866935 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 164 with MSE 0.7689455605633688 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 165 with MSE 0.7668551979443827 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 166 with MSE 0.7573989042644808 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 167 with MSE 0.7544093953889386 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 168 with MSE 0.765114632602041 

order: 1 , 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 235 with MSE 0.7318471319504887 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 236 with MSE 0.742491301768388 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 237 with MSE 0.7350743927056405 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 238 with MSE 0.727712126513259 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 239 with MSE 0.7363007022810456 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 240 with MSE 0.7443088805131199 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 241 with MSE 0.7397565802819326 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 242 with MSE 0.7378909897024981 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 243 with MSE 0.7353091390876773 

order: 1 , l

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 314 with MSE 0.7308140772653696 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 315 with MSE 0.7413982228104319 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 316 with MSE 0.7319794067161918 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 317 with MSE 0.723727227573517 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 318 with MSE 0.7219309166501342 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 319 with MSE 0.7341742558498103 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 320 with MSE 0.7209121131193144 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 321 with MSE 0.7311341013001788 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 322 with MSE 0.7403300534235947 

order: 1 , 

...

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8302 with MSE 0.7123844872414283 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8303 with MSE 0.7134420352523064 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8304 with MSE 0.7161932311973516 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8305 with MSE 0.6987246336402847 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8306 with MSE 0.7081394559486829 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8307 with MSE 0.7097447032306813 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8308 with MSE 0.7196478045069947 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8309 with MSE 0.7096223501044938 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8310 with MSE 0.7045362663363295 

o

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8378 with MSE 0.7076733350921905 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8379 with MSE 0.7135776110380891 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8380 with MSE 0.7130870795963551 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8381 with MSE 0.7104493243190898 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8382 with MSE 0.699657180362145 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8383 with MSE 0.721756832426252 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8384 with MSE 0.7049505223087535 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8385 with MSE 0.7069646620103008 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8386 with MSE 0.7183478544189269 

ord

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8455 with MSE 0.7081697086056435 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8456 with MSE 0.7083272089760996 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8457 with MSE 0.7075152018524923 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8458 with MSE 0.7035216500600499 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8459 with MSE 0.7171176293438731 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8460 with MSE 0.7070581252278478 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8461 with MSE 0.7158051865097351 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8462 with MSE 0.7172441773896583 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8463 with MSE 0.7083018161279778 

o

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8529 with MSE 0.7110838041272667 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8530 with MSE 0.7175366202087077 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8531 with MSE 0.7165556558577391 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8532 with MSE 0.709443659430537 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8533 with MSE 0.7031251094287037 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8534 with MSE 0.7121442168752448 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8535 with MSE 0.7161146852405129 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8536 with MSE 0.7084095562607965 

order: 1 , learning rate: 1e-05 , regularizer: 0.1 
Convergence after epoch 8537 with MSE 0.7056475200427106 

or

KeyboardInterrupt: 

Complete the following function which selects the best hyperparameter combination (i.e. the one that gives lowest MSE on **validation data**). (0.5 point)

In [ ]:
# find hparams of minimum MSE on Validation data
def find_best_hparams(weights_hist):
    # TODO: implement
    hpm_best, mse_best = None
    return hpm_best, mse_best

best_hpm_combination = find_best_hparams(weights_hist)

###### 9. Re-Training on Train + Validation data
Complete the following function which does re-training on the combined training and validation data. (**1 point**)

In [2]:
# re-run the training on X_train + X_val combined
# Later test it on X_test; That will be our best possible MSE on test data
# this will be more or less the same training code as you did above
# but, here we just have only one value for each hyperparameter.

# TODO: implement

In [4]:
# plot the convergence of MSE values using matplotlib
# i.e. #epochs on X-axis and MSE values on Y-axis
# TODO: implement

###### 10. Evaluation on Test set
Evaluate your model on test data. (1.0 point)

**Please note that you should keep X_test undisturbed throughout this whole phase.** Else restart the kernel and start from beginning. The whole point of this exercise would not make sense if test data has been *seen in training*.

In [ ]:
# finally!!!
# test it on X_test with the Weight vector that you found above
# this will be the generalization error of our model!!
# TODO: implement

#print("Finally!!! MSE achieved on X_test is : {}".format(round(mse_test, 6)))

###### 11. Results
Please report the following

a) MSE value on Test data. (0.5 points)

b) Which hyperparameter combination turned out to be the best? In your understanding, why do you think such a combination turned out to be the best for this task? (1.0 point)

# Bonus (2 points)

Now, please repeat the whole *training, validation, re-training, and testing* procedure that we talked about above with the following hyperparameter combination:

In [2]:
polynomial_order = [1]
learning_rates = [0.1]
lambdas = [0.1]

What are your observations during the training phase? Please explain why such a behaviour happened.

---

## Submission instructions
You should provide a single Jupyter notebook as the solution. The naming should include the assignment number and matriculation IDs of all members in your team in the following format:
**assignment-4_matriculation1_matriculation2_matriculation3.ipynb** (in case of 3 members in a team). 
Make sure to keep the order matriculation1_matriculation2_matriculation3 the same for all assignments.

Please submit the solution to your tutor (with **[NNIA][assignment-4]** in email subject):
1. Maksym Andriushchenko <s8mmandr@stud.uni-saarland.de>
2. Marius Mosbach <s9msmosb@stud.uni-saarland.de>
3. Rajarshi Biswas <rbisw17@gmail.com>
4. Marimuthu Kalimuthu <s8makali@stud.uni-saarland.de>

Note: **If you are in a team, please submit only 1 solution to only 1 tutor.**